### Extract scientific abstracts
Dataset is from the "A typology of scientific breakthrough" paper

In [114]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup  
from tqdm import tqdm
import requests
import time
import re

In [189]:
df = pd.read_excel('./TypologyOfBreakthroughs_dataset.xlsx')

pubmed_urls = []
pubmed_responses = []
abstracts = ['']*len(df)


#pubmed search based approach
for index,row in df.iterrows():
    doi = row['DOI']
    pubmed_url = f'https://pubmed.ncbi.nlm.nih.gov/?term={doi}'
    pubmed_urls.append(pubmed_url)


for url in tqdm(pubmed_urls):
    pubmed_response = requests.get(url)
    pubmed_responses.append(pubmed_response)
    time.sleep(0.5)


for i in tqdm(range(len(pubmed_responses))): 
    soup = BeautifulSoup(pubmed_responses[i].content, 'html.parser')
    web_title = soup.title.string
    if web_title[:2] == '10':
        abstracts[i] = 'NA'
        continue
    abstract = soup.find('div', class_='abstract-content selected')
    if abstract is None:
        abstracts[i] = 'NA'
        continue
    cleaned_text = re.sub(r'\s+', ' ', abstract.text).strip()
    abstracts[i] = cleaned_text


df['Abstract'] = abstracts
df.to_csv('TypologyOfBreakthroughs_dataset_with_abstracts.csv', index=False)

100%|██████████| 335/335 [00:17<00:00, 18.70it/s]
